In [289]:
import pandas as pd
import os
import re

In [290]:
data = pd.read_csv('/Users/briannahancock/Desktop/Final_Data/movies_metadata.csv')
data.head()

/Users/briannahancock/opt/anaconda3/envs/PythonData2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [291]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [292]:
# Dropping columns with lots of null rows, and JSON data types
data = data.drop(columns=['belongs_to_collection', 'homepage', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'tagline'])

In [293]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  object 
 2   genres             45466 non-null  object 
 3   id                 45466 non-null  object 
 4   imdb_id            45449 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   popularity         45461 non-null  object 
 8   poster_path        45080 non-null  object 
 9   release_date       45379 non-null  object 
 10  revenue            45460 non-null  float64
 11  runtime            45203 non-null  float64
 12  status             45379 non-null  object 
 13  title              45460 non-null  object 
 14  video              45460 non-null  object 
 15  vote_average       45460 non-null  float64
 16  vote_count         454

In [294]:
data.columns

Index(['adult', 'budget', 'genres', 'id', 'imdb_id', 'original_language',
       'original_title', 'popularity', 'poster_path', 'release_date',
       'revenue', 'runtime', 'status', 'title', 'video', 'vote_average',
       'vote_count'],
      dtype='object')

In [295]:
imdb = pd.Series(data.imdb_id)
imdb

0        tt0114709
1        tt0113497
2        tt0113228
3        tt0114885
4        tt0113041
           ...    
45461    tt6209470
45462    tt2028550
45463    tt0303758
45464    tt0008536
45465    tt6980792
Name: imdb_id, Length: 45466, dtype: object

In [296]:
# Cleaning up imdb id to map to other tables
new_imdb = []
for id in imdb:
    # print(id)
    id = str(id)
    id = id.replace('tt', '')
    new_imdb.append(id)
new_imdb

['0114709',
 '0113497',
 '0113228',
 '0114885',
 '0113041',
 '0113277',
 '0114319',
 '0112302',
 '0114576',
 '0113189',
 '0112346',
 '0112896',
 '0112453',
 '0113987',
 '0112760',
 '0112641',
 '0114388',
 '0113101',
 '0112281',
 '0113845',
 '0113161',
 '0112722',
 '0112401',
 '0114168',
 '0113627',
 '0114057',
 '0114011',
 '0114117',
 '0112682',
 '0115012',
 '0112792',
 '0114746',
 '0114952',
 '0112431',
 '0112637',
 '0112818',
 '0112286',
 '0113442',
 '0112697',
 '0112749',
 '0114279',
 '0112819',
 '0114272',
 '0113855',
 '0114681',
 '0113347',
 '0114369',
 '0114148',
 '0114916',
 '0114814',
 '0109950',
 '0113819',
 '0110299',
 '0112499',
 '0113158',
 '0113541',
 '0113321',
 '0110877',
 '0112714',
 '0113419',
 '0116260',
 '0113862',
 '0116126',
 '0118002',
 '0115683',
 '0116839',
 '0114753',
 '0113149',
 '0113118',
 '0116367',
 '0113010',
 '0113537',
 '0113828',
 '0115644',
 '0115676',
 '0114367',
 '0113973',
 '0112744',
 '0116731',
 '0112445',
 '0114660',
 '0112379',
 '0114039',
 '01

In [297]:
data['imdb_id'] = new_imdb

In [298]:
data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [299]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45466 non-null  object 
 1   budget             45466 non-null  object 
 2   genres             45466 non-null  object 
 3   id                 45466 non-null  object 
 4   imdb_id            45466 non-null  object 
 5   original_language  45455 non-null  object 
 6   original_title     45466 non-null  object 
 7   popularity         45461 non-null  object 
 8   poster_path        45080 non-null  object 
 9   release_date       45379 non-null  object 
 10  revenue            45460 non-null  float64
 11  runtime            45203 non-null  float64
 12  status             45379 non-null  object 
 13  title              45460 non-null  object 
 14  video              45460 non-null  object 
 15  vote_average       45460 non-null  float64
 16  vote_count         454

In [300]:
data['id'].nunique()

45436

In [301]:
data['original_language'].nunique()

92

In [302]:
data['original_language'].value_counts()

en       32269
fr        2438
it        1529
ja        1350
de        1080
         ...  
zu           1
qu           1
104.0        1
la           1
si           1
Name: original_language, Length: 92, dtype: int64

In [303]:
# Dropping duplicate movie id's for primary key constraint
data = data.drop_duplicates(subset='id', keep='first')

In [304]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45436 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45436 non-null  object 
 1   budget             45436 non-null  object 
 2   genres             45436 non-null  object 
 3   id                 45436 non-null  object 
 4   imdb_id            45436 non-null  object 
 5   original_language  45425 non-null  object 
 6   original_title     45436 non-null  object 
 7   popularity         45431 non-null  object 
 8   poster_path        45050 non-null  object 
 9   release_date       45349 non-null  object 
 10  revenue            45430 non-null  float64
 11  runtime            45173 non-null  float64
 12  status             45349 non-null  object 
 13  title              45430 non-null  object 
 14  video              45430 non-null  object 
 15  vote_average       45430 non-null  float64
 16  vote_count         454

In [305]:
# Dropping rows with bad data
data = data.drop(labels=[19722,19723,29483,29484,35564,35565], axis=0)

In [306]:
data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [307]:
df = data.explode(['genres'])
df.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,"[{'id': 35, 'name': 'Comedy'}]",11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [308]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45430 entries, 0 to 45465
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              45430 non-null  object 
 1   budget             45430 non-null  object 
 2   genres             45430 non-null  object 
 3   id                 45430 non-null  object 
 4   imdb_id            45430 non-null  object 
 5   original_language  45419 non-null  object 
 6   original_title     45430 non-null  object 
 7   popularity         45425 non-null  object 
 8   poster_path        45044 non-null  object 
 9   release_date       45343 non-null  object 
 10  revenue            45424 non-null  float64
 11  runtime            45167 non-null  float64
 12  status             45343 non-null  object 
 13  title              45424 non-null  object 
 14  video              45424 non-null  object 
 15  vote_average       45424 non-null  float64
 16  vote_count         454

In [309]:
data['genres'] = data['genres'].str.findall("(?:'name':\s)(\W[A-Z][a-z]*\W)(?:})")
data.head()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
0,False,30000000,"['Animation', 'Comedy', 'Family']",862,0114709,en,Toy Story,21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,Toy Story,False,7.7,5415.0
1,False,65000000,"['Adventure', 'Fantasy', 'Family']",8844,0113497,en,Jumanji,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Jumanji,False,6.9,2413.0
2,False,0,"['Romance', 'Comedy']",15602,0113228,en,Grumpier Old Men,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Grumpier Old Men,False,6.5,92.0
3,False,16000000,"['Comedy', 'Drama', 'Romance']",31357,0114885,en,Waiting to Exhale,3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Waiting to Exhale,False,6.1,34.0
4,False,0,['Comedy'],11862,0113041,en,Father of the Bride Part II,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Father of the Bride Part II,False,5.7,173.0


In [310]:
from ast import literal_eval
ind, value = zip(*[(i, x) 
                  for i, row in data['genres'].items() 
                  for x in literal_eval(row)])
genre_col = pd.DataFrame(value, index=ind)
genre_col

ValueError: malformed node or string: ["'Animation'", "'Comedy'", "'Family'"]

In [ ]:
data.tail()

,adult,budget,genres,id,imdb_id,original_language,original_title,popularity,poster_path,release_date,revenue,runtime,status,title,video,vote_average,vote_count
45461,False,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",439050,6209470,fa,رگ خواب,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,NaN,0.0,90.0,Released,Subdue,False,4.0,1.0
45462,False,0,"[{'id': 18, 'name': 'Drama'}]",111109,2028550,tl,Siglo ng Pagluluwal,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,2011-11-17,0.0,360.0,Released,Century of Birthing,False,9.0,3.0
45463,False,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",67758,0303758,en,Betrayal,0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,2003-08-01,0.0,90.0,Released,Betrayal,False,3.8,6.0
45464,False,0,[],227506,0008536,en,Satana likuyushchiy,0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,1917-10-21,0.0,87.0,Released,Satan Triumphant,False,0.0,0.0
45465,False,0,[],461257,6980792,en,Queerama,0.163015,/s5UkZt6NTsrS7ZF0Rh8nzupRlIU.jpg,2017-06-09,0.0,75.0,Released,Queerama,False,0.0,0.0


In [ ]:
data = data.merge(genre_col, how='left', left_index=True, right_index=True)

In [ ]:
data.columns

In [ ]:
data['name']

In [ ]:
data.isna().count()

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
titles = data['original_title']
cleaned_titles = []


In [ ]:
data.to_csv('/Users/briannahancock/Desktop/Final_Data/movies_metadata_cleaned.csv')